#  Load first table from the page into df dataframe.  #

# Toronto Data # 

### Import necessary Libraries


In [15]:
import numpy as np  
import pandas as pd 
import matplotlib.pyplot
import seaborn as sns 
import folium   
import os 
import json    
from geopy.geocoders import Nominatim  
import requests 
from pandas.io.json import json_normalize  
import matplotlib.cm as cm
import matplotlib.colors as colors 
from sklearn.cluster import KMeans   
import xlwt
#!pip install xlwt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)   
os.getcwd()
path = os.getcwd() 
 

print('Libraries imported.')    

Libraries imported.


In [4]:
df1=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',header=0  )
type(df1)
neighborhd=df1[0]
neighborhd=neighborhd.drop([0])
 

neighborhd.rename(columns={0:'Postal Code', 1: 'Borough', 2: 'Neighborhood'}, inplace=True)

neighborhd=neighborhd[neighborhd.Borough !='Not assigned']
neighborhd=pd.DataFrame(neighborhd)

neighborhd.set_index(['Postal Code','Borough'],inplace=True)
merge = neighborhd.groupby(level=['Postal Code','Borough'], sort=False).agg( ','.join)
 
serial=merge.reset_index()
serial.loc[4, 'Neighbourhood']='Central\'s Ave'
serial.to_excel('project_table.xls')

df=pd.DataFrame(serial)
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,Central's Ave


In [39]:
df2=pd.read_csv('https://cocl.us/Geospatial_data') 

frames=[df,df2]
frames=pd.concat(frames, axis=1, sort=False)

merge_columns=pd.merge(df, df2, left_on='Postal Code', right_on='Postal Code')

merge_columns.to_csv('neigbors_geographical.csv')
merge_columns.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,Central's Ave,43.662301,-79.389494


In [40]:
downtown_toronto = merge_columns[merge_columns['Borough'] == 'Downtown Toronto'].reset_index(drop=True)

downtown_toronto= downtown_toronto.drop(['Postal Code'], axis=1)
downtown_toronto.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,Downtown Toronto,Central's Ave,43.662301,-79.389494
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,Downtown Toronto,St. James Town,43.651494,-79.375418
4,Downtown Toronto,Berczy Park,43.644771,-79.373306


# New York Data # 

In [6]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
neighborhoods_data = newyork_data['features']

neighborhoods_data[0]

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)



Data downloaded!


In [7]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods.head()
    

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [8]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(  len(neighborhoods['Borough'].unique()),  neighborhoods.shape[0]))
  
neighborhoods.head()

The dataframe has 5 boroughs and 306 neighborhoods.


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [9]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()



,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [10]:
CLIENT_ID = '33ACTML2RLMHBCJWGG2OZ223S4T4RUPIQMTPBO4TUEHY5T4M'  
CLIENT_SECRET = 'SZHA2R02NBX30DS5WT1PSYI3NKXL2OU1VE112DNRI2C03FYE'  
VERSION = '20180604'
LIMIT = 20
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 33ACTML2RLMHBCJWGG2OZ223S4T4RUPIQMTPBO4TUEHY5T4M
CLIENT_SECRET:SZHA2R02NBX30DS5WT1PSYI3NKXL2OU1VE112DNRI2C03FYE


In [11]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude_downtown_manhattan = location.latitude
longitude_downtown_manhattan = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude_downtown_manhattan, longitude_downtown_manhattan))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [42]:
address = 'Downtown Toronto, ON, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude_downtown_toronto = location.latitude
longitude_downtown_toronto = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude_downtown_toronto, longitude_downtown_toronto))

The geograpical coordinate of Toronto City are 43.6563221, -79.3809161.


#  Visualization of data # 

# Toronto Maps #  

In [43]:
map_downtown_toronto = folium.Map(location=[latitude_downtown_toronto,longitude_downtown_toronto], zoom_start=11)

# add markers to map
for lat, lng, label in zip(downtown_toronto['Latitude'], downtown_toronto['Longitude'], downtown_toronto['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown_toronto)  
    
map_downtown_toronto


In [44]:
from folium import plugins

map_downtown_toronto = folium.Map(location=[latitude_downtown_toronto,longitude_downtown_toronto], zoom_start=11)

incidents = plugins.MarkerCluster().add_to(map_downtown_toronto)

for lat, lng, label in zip(downtown_toronto['Latitude'], downtown_toronto['Longitude'], downtown_toronto['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(incidents)  
    
map_downtown_toronto

# Manhattan Maps #  

In [12]:
map_manhattan = folium.Map(location=[latitude_downtown_manhattan, longitude_downtown_manhattan], zoom_start=11)

for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [17]:
map_manhattan = folium.Map(location=[latitude_downtown_manhattan, longitude_downtown_manhattan], zoom_start=11)
import folium.plugins as plugins

grouping = plugins.MarkerCluster().add_to(map_manhattan)

for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(grouping)  
    
map_manhattan

# Analysis of data #

# Exploring Neighborhoods in Downtown Toronto #


In [37]:
def getNearbyVenues(names, latitudes,longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names,latitudes,longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [45]:
 
downtown_toronto_venues = getNearbyVenues(names=downtown_toronto['Neighbourhood'],
                                   latitudes=downtown_toronto['Latitude'],
                                   longitudes=downtown_toronto['Longitude'])

Regent Park, Harbourfront
Central's Ave
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [46]:
# Let's check the size of the resulting dataframe
print(downtown_toronto_venues.shape)
downtown_toronto_venues.head()

(356, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [47]:
downtown_toronto_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,20,20,20,20,20,20
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,20,20,20,20,20,20
Central's Ave,20,20,20,20,20,20
Christie,16,16,16,16,16,16
Church and Wellesley,20,20,20,20,20,20
"Commerce Court, Victoria Hotel",20,20,20,20,20,20
"First Canadian Place, Underground city",20,20,20,20,20,20
"Garden District, Ryerson",20,20,20,20,20,20


In [48]:
print('There are {} uniques categories.'.format(len(downtown_toronto_venues['Venue Category'].unique())))

There are 125 uniques categories.


In [50]:
downtown_toronto_onehot = pd.get_dummies(downtown_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_toronto_onehot['Neighborhood'] = downtown_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_toronto_onehot.columns[-1]] + list(downtown_toronto_onehot.columns[:-1])
downtown_toronto_onehot = downtown_toronto_onehot[fixed_columns]

downtown_toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Distribution Center,Electronics Store,Farmers Market,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Gastropub,General Entertainment,Gift Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,Hobby Shop,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Liquor Store,Lounge,Martial Arts School,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Museum,Music Venue,Neighborhood,Nightclub,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pizza Place,Plane,Playground,Plaza,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [51]:
downtown_toronto_grouped = downtown_toronto_onehot.groupby('Neighborhood').mean().reset_index()

num_top_venues = 5

for hood in downtown_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_toronto_grouped[downtown_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.10
1  Seafood Restaurant  0.10
2      Farmers Market  0.10
3            Beer Bar  0.10
4   French Restaurant  0.05


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
             venue  freq
0   Airport Lounge  0.12
1  Airport Service  0.12
2            Plane  0.06
3              Bar  0.06
4    Boat or Ferry  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.20
1  Italian Restaurant  0.10
2                 Spa  0.05
3            Tea Room  0.05
4  Miscellaneous Shop  0.05


----Central's Ave----
                 venue  freq
0          Coffee Shop  0.20
1          Yoga Studio  0.05
2                 Bank  0.05
3   Italian Restaurant  0.05
4  Fried Chicken Joint  0.05


----Christie----
           venue  freq
0  Grocery Store  0.25
1           Café  0.19
2           Park  0.12
3    Candy Store  0.06
4      Nightclub

In [52]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_toronto_grouped['Neighborhood']

for ind in np.arange(downtown_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Farmers Market,Seafood Restaurant,Coffee Shop,Beer Bar,Bakery,Liquor Store,Concert Hall,Park,Restaurant,Cocktail Bar
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Plane,Harbor / Marina,Coffee Shop,Rental Car Location,Boutique,Boat or Ferry,Sculpture Garden,Bar
2,Central Bay Street,Coffee Shop,Italian Restaurant,Japanese Restaurant,Bubble Tea Shop,Seafood Restaurant,Spa,Ramen Restaurant,Art Museum,Sushi Restaurant,Tea Room
3,Central's Ave,Coffee Shop,Yoga Studio,Smoothie Shop,Hobby Shop,Distribution Center,Diner,Italian Restaurant,Creperie,Mexican Restaurant,Chinese Restaurant
4,Christie,Grocery Store,Café,Park,Coffee Shop,Athletics & Sports,Candy Store,Nightclub,Italian Restaurant,Baby Store,Restaurant
5,Church and Wellesley,Pizza Place,Creperie,Park,Café,Coffee Shop,Mexican Restaurant,Burger Joint,Bubble Tea Shop,Breakfast Spot,Martial Arts School
6,"Commerce Court, Victoria Hotel",Café,Gastropub,Japanese Restaurant,Coffee Shop,Restaurant,Bakery,Gym / Fitness Center,Hotel,Museum,Pub
7,"First Canadian Place, Underground city",Café,Coffee Shop,Restaurant,Gym,Steakhouse,Pizza Place,Gastropub,Pub,Seafood Restaurant,Bakery
8,"Garden District, Ryerson",Café,Clothing Store,Pizza Place,Diner,Burrito Place,Burger Joint,Plaza,Ramen Restaurant,Coffee Shop,College Rec Center
9,"Harbourfront East, Union Station, Toronto Islands",Park,Café,Plaza,Skating Rink,Salad Place,Japanese Restaurant,Bubble Tea Shop,Deli / Bodega,Sporting Goods Shop,Ice Cream Shop



# Clustering Neighborhoods #


In [53]:
kclusters = 5

downtown_toronto_grouped_clustering = downtown_toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_toronto_grouped_clustering)

kmeans.labels_[0:10] 

array([1, 1, 4, 4, 0, 1, 3, 3, 1, 1], dtype=int32)

In [54]:
downtown_toronto_merged = downtown_toronto

downtown_toronto_merged['Cluster Labels'] = kmeans.labels_

downtown_toronto_merged = downtown_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

downtown_toronto_merged.head()

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Park,Breakfast Spot,Distribution Center,Chocolate Shop,Pub,Restaurant,Performing Arts Venue,Spa,Dessert Shop
1,Downtown Toronto,Central's Ave,43.662301,-79.389494,1,Coffee Shop,Yoga Studio,Smoothie Shop,Hobby Shop,Distribution Center,Diner,Italian Restaurant,Creperie,Mexican Restaurant,Chinese Restaurant
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,4,Café,Clothing Store,Pizza Place,Diner,Burrito Place,Burger Joint,Plaza,Ramen Restaurant,Coffee Shop,College Rec Center
3,Downtown Toronto,St. James Town,43.651494,-79.375418,4,Gastropub,Japanese Restaurant,Coffee Shop,Restaurant,BBQ Joint,Food Truck,Hotel,Italian Restaurant,Creperie,Cosmetics Shop
4,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Farmers Market,Seafood Restaurant,Coffee Shop,Beer Bar,Bakery,Liquor Store,Concert Hall,Park,Restaurant,Cocktail Bar


In [55]:
map_clusters = folium.Map(location=[latitude_downtown_toronto, longitude_downtown_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_toronto_merged['Latitude'], downtown_toronto_merged['Longitude'], downtown_toronto_merged['Neighbourhood'], downtown_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Analyse Clusters

# Cluster 1 (Restaurants, Cafeterias, Supermarket, etc) #

In [61]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 0, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Berczy Park,Farmers Market,Seafood Restaurant,Coffee Shop,Beer Bar,Bakery,Liquor Store,Concert Hall,Park,Restaurant,Cocktail Bar


# Cluster 2 # 

In [57]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 1, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",Coffee Shop,Park,Breakfast Spot,Distribution Center,Chocolate Shop,Pub,Restaurant,Performing Arts Venue,Spa,Dessert Shop
1,Central's Ave,Coffee Shop,Yoga Studio,Smoothie Shop,Hobby Shop,Distribution Center,Diner,Italian Restaurant,Creperie,Mexican Restaurant,Chinese Restaurant
5,Central Bay Street,Coffee Shop,Italian Restaurant,Japanese Restaurant,Bubble Tea Shop,Seafood Restaurant,Spa,Ramen Restaurant,Art Museum,Sushi Restaurant,Tea Room
8,"Harbourfront East, Union Station, Toronto Islands",Park,Café,Plaza,Skating Rink,Salad Place,Japanese Restaurant,Bubble Tea Shop,Deli / Bodega,Sporting Goods Shop,Ice Cream Shop
9,"Toronto Dominion Centre, Design Exchange",Coffee Shop,Café,Restaurant,Japanese Restaurant,Gym,Hotel,Gastropub,Pub,Beer Bar,Steakhouse
10,"Commerce Court, Victoria Hotel",Café,Gastropub,Japanese Restaurant,Coffee Shop,Restaurant,Bakery,Gym / Fitness Center,Hotel,Museum,Pub
15,Stn A PO Boxes,Cocktail Bar,Beer Bar,Farmers Market,French Restaurant,Japanese Restaurant,Café,Museum,Restaurant,Concert Hall,Seafood Restaurant
16,"St. James Town, Cabbagetown",Café,Restaurant,Pet Store,Indian Restaurant,Park,Caribbean Restaurant,Butcher,Pub,Deli / Bodega,Bakery
18,Church and Wellesley,Pizza Place,Creperie,Park,Café,Coffee Shop,Mexican Restaurant,Burger Joint,Bubble Tea Shop,Breakfast Spot,Martial Arts School


# Cluster 3 # 

In [58]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 2, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Plane,Harbor / Marina,Coffee Shop,Rental Car Location,Boutique,Boat or Ferry,Sculpture Garden,Bar


# Cluster 4 # 

In [59]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 3, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Christie,Grocery Store,Café,Park,Coffee Shop,Athletics & Sports,Candy Store,Nightclub,Italian Restaurant,Baby Store,Restaurant
7,"Richmond, Adelaide, King",Seafood Restaurant,Steakhouse,Café,Pizza Place,Lounge,Plaza,Coffee Shop,Restaurant,Concert Hall,Smoke Shop
12,"Kensington Market, Chinatown, Grange Park",Café,Vietnamese Restaurant,Wine Bar,Bakery,Fish Market,Farmers Market,Dessert Shop,Mexican Restaurant,Coffee Shop,Organic Grocery
14,Rosedale,Park,Playground,Trail,Creperie,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop
17,"First Canadian Place, Underground city",Café,Coffee Shop,Restaurant,Gym,Steakhouse,Pizza Place,Gastropub,Pub,Seafood Restaurant,Bakery


# Cluster 5 # 

In [60]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 3, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Christie,Grocery Store,Café,Park,Coffee Shop,Athletics & Sports,Candy Store,Nightclub,Italian Restaurant,Baby Store,Restaurant
7,"Richmond, Adelaide, King",Seafood Restaurant,Steakhouse,Café,Pizza Place,Lounge,Plaza,Coffee Shop,Restaurant,Concert Hall,Smoke Shop
12,"Kensington Market, Chinatown, Grange Park",Café,Vietnamese Restaurant,Wine Bar,Bakery,Fish Market,Farmers Market,Dessert Shop,Mexican Restaurant,Coffee Shop,Organic Grocery
14,Rosedale,Park,Playground,Trail,Creperie,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop
17,"First Canadian Place, Underground city",Café,Coffee Shop,Restaurant,Gym,Steakhouse,Pizza Place,Gastropub,Pub,Seafood Restaurant,Bakery


# Exploring Neighborhoods in Manhattan #


In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
# Now write the code to run the above function on each neighborhood and create a new dataframe called manhattan_venues
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude'],
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [22]:
manhattan_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,20,20,20,20,20,20
Carnegie Hill,20,20,20,20,20,20
Central Harlem,20,20,20,20,20,20
Chelsea,20,20,20,20,20,20
Chinatown,20,20,20,20,20,20
Civic Center,20,20,20,20,20,20
Clinton,20,20,20,20,20,20
East Harlem,20,20,20,20,20,20
East Village,20,20,20,20,20,20


In [23]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))


There are 211 uniques categories.


Analyzing neighborgs

In [24]:
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Duty-free Shop,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Heliport,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Leather Goods Store,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,New American Restaurant,Noodle House,Opera House,Outdoor Sculpture,Outdoors & Recreation,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pub,Public Art,Ramen Restaurant,Residential Building (Apartment / Condo),Restaurant,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Club,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Tourist Information Center,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Marble Hill,0,0,0,0,

In [25]:
# Set Index
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()

num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
           venue  freq
0  Memorial Site  0.15
1           Park  0.15
2     Food Court  0.10
3     Auditorium  0.05
4  Shopping Mall  0.05


----Carnegie Hill----
                  venue  freq
0           Coffee Shop  0.10
1  Gym / Fitness Center  0.10
2    Italian Restaurant  0.10
3                   Gym  0.10
4            Shoe Store  0.05


----Central Harlem----
                venue  freq
0   French Restaurant  0.10
1  African Restaurant  0.10
2           Jazz Club  0.05
3        Cocktail Bar  0.05
4            Boutique  0.05


----Chelsea----
                venue  freq
0             Theater  0.10
1   French Restaurant  0.05
2    Tapas Restaurant  0.05
3         Fish Market  0.05
4  Chinese Restaurant  0.05


----Chinatown----
                venue  freq
0  Chinese Restaurant  0.15
1      Sandwich Place  0.10
2              Museum  0.05
3        Noodle House  0.05
4    Greek Restaurant  0.05


----Civic Center----
                 venue  freq
0            

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Memorial Site,Food Court,Auditorium,Sandwich Place,Building,Shopping Mall,Coffee Shop,Gym,Scenic Lookout
1,Carnegie Hill,Coffee Shop,Gym / Fitness Center,Gym,Italian Restaurant,Pizza Place,Café,Dance Studio,French Restaurant,Community Center,Shoe Store
2,Central Harlem,African Restaurant,French Restaurant,Music Venue,Ethiopian Restaurant,Café,Bar,Juice Bar,Bagel Shop,Boutique,Jazz Club
3,Chelsea,Theater,Speakeasy,Hotel,Coffee Shop,New American Restaurant,Chinese Restaurant,Café,Fish Market,French Restaurant,Sushi Restaurant
4,Chinatown,Chinese Restaurant,Sandwich Place,Greek Restaurant,Noodle House,Bubble Tea Shop,Museum,Bike Shop,Salon / Barbershop,New American Restaurant,Cocktail Bar
5,Civic Center,Spa,Yoga Studio,Gym,Cuban Restaurant,Dance Studio,Falafel Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Burrito Place,Bubble Tea Shop
6,Clinton,Gym / Fitness Center,Theater,Hotel,Pizza Place,Peruvian Restaurant,Park,Comedy Club,Coffee Shop,Movie Theater,Café
7,East Harlem,Mexican Restaurant,Thai Restaurant,Bakery,Cuban Restaurant,Steakhouse,French Restaurant,Donut Shop,Taco Place,New American Restaurant,Beer Bar
8,East Village,Korean Restaurant,Vietnamese Restaurant,Pet Café,Beer Store,Dog Run,Beer Bar,Bar,Moroccan Restaurant,Bagel Shop,Scandinavian Restaurant
9,Financial District,Coffee Shop,Gym / Fitness Center,Salad Place,Event Space,Pizza Place,Steakhouse,Café,New American Restaurant,Doctor's Office,Falafel Restaurant


Now we are going to cluster Neighborhood in Manhattan

In [27]:
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 4, 3, 3, 4, 3, 3, 1, 1, 4], dtype=int32)

In [28]:
manhattan_merged = manhattan_data

# add clustering labels
manhattan_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1,Coffee Shop,Gym,Sandwich Place,Discount Store,Supplement Shop,Ice Cream Shop,Pizza Place,Diner,Tennis Stadium,Seafood Restaurant
1,Manhattan,Chinatown,40.715618,-73.994279,4,Chinese Restaurant,Sandwich Place,Greek Restaurant,Noodle House,Bubble Tea Shop,Museum,Bike Shop,Salon / Barbershop,New American Restaurant,Cocktail Bar
2,Manhattan,Washington Heights,40.851903,-73.936900,3,Café,Wine Shop,Park,Breakfast Spot,Italian Restaurant,Market,New American Restaurant,Coffee Shop,Restaurant,Ramen Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,3,Bakery,Deli / Bodega,Park,Yoga Studio,Pet Store,Diner,Café,Farmers Market,Mexican Restaurant,Bistro
4,Manhattan,Hamilton Heights,40.823604,-73.949688,4,Yoga Studio,Mexican Restaurant,Cocktail Bar,Bakery,Smoke Shop,Café,Mediterranean Restaurant,Caribbean Restaurant,Coffee Shop,Burger Joint


In [30]:
# create map
map_clusters = folium.Map(location=[latitude_downtown_manhattan, longitude_downtown_manhattan], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Cluster 1 (Commercial) #

In [31]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Chelsea,Theater,Speakeasy,Hotel,Coffee Shop,New American Restaurant,Chinese Restaurant,Café,Fish Market,French Restaurant,Sushi Restaurant


# Cluster 2 (Residential) #

In [32]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Coffee Shop,Gym,Sandwich Place,Discount Store,Supplement Shop,Ice Cream Shop,Pizza Place,Diner,Tennis Stadium,Seafood Restaurant
7,East Harlem,Mexican Restaurant,Thai Restaurant,Bakery,Cuban Restaurant,Steakhouse,French Restaurant,Donut Shop,Taco Place,New American Restaurant,Beer Bar
8,Upper East Side,Hotel,Bakery,Pizza Place,Cosmetics Shop,Hotel Bar,Park,Coffee Shop,Sandwich Place,Chocolate Shop,Burrito Place
15,Midtown,Hotel,Cuban Restaurant,Bookstore,Sporting Goods Shop,Spa,Smoke Shop,Salad Place,Clothing Store,Cycle Studio,Szechuan Restaurant
20,Lower East Side,Coffee Shop,Chinese Restaurant,Art Gallery,Japanese Restaurant,Yoga Studio,Juice Bar,Performing Arts Venue,Cocktail Bar,Filipino Restaurant,Café
25,Manhattan Valley,Bar,Yoga Studio,Thai Restaurant,Italian Restaurant,Hawaiian Restaurant,Grocery Store,Korean Restaurant,Fried Chicken Joint,Mexican Restaurant,Park
28,Battery Park City,Park,Memorial Site,Food Court,Auditorium,Sandwich Place,Building,Shopping Mall,Coffee Shop,Gym,Scenic Lookout
30,Carnegie Hill,Coffee Shop,Gym / Fitness Center,Gym,Italian Restaurant,Pizza Place,Café,Dance Studio,French Restaurant,Community Center,Shoe Store
32,Civic Center,Spa,Yoga Studio,Gym,Cuban Restaurant,Dance Studio,Falafel Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Burrito Place,Bubble Tea Shop
33,Midtown South,Korean Restaurant,Coffee Shop,Dessert Shop,Snack Place,Shoe Store,Boutique,Lingerie Store,Leather Goods Store,Clothing Store,Grocery Store


# Cluster 3 (Center area) #

In [34]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Café,Wine Shop,Park,Breakfast Spot,Italian Restaurant,Market,New American Restaurant,Coffee Shop,Restaurant,Ramen Restaurant
3,Inwood,Bakery,Deli / Bodega,Park,Yoga Studio,Pet Store,Diner,Café,Farmers Market,Mexican Restaurant,Bistro
5,Manhattanville,Coffee Shop,Italian Restaurant,Café,BBQ Joint,Bike Trail,Gastropub,Dumpling Restaurant,Lounge,Supermarket,Bar
6,Central Harlem,African Restaurant,French Restaurant,Music Venue,Ethiopian Restaurant,Café,Bar,Juice Bar,Bagel Shop,Boutique,Jazz Club
10,Lenox Hill,Gym,Thai Restaurant,Gift Shop,Middle Eastern Restaurant,Smoke Shop,Liquor Store,Taco Place,Salad Place,Dessert Shop,Gourmet Shop
14,Clinton,Gym / Fitness Center,Theater,Hotel,Pizza Place,Peruvian Restaurant,Park,Comedy Club,Coffee Shop,Movie Theater,Café
23,Soho,Clothing Store,Sporting Goods Shop,Shoe Store,Men's Store,Dance Studio,Salon / Barbershop,Boutique,Miscellaneous Shop,Cycle Studio,Supermarket
29,Financial District,Coffee Shop,Gym / Fitness Center,Salad Place,Event Space,Pizza Place,Steakhouse,Café,New American Restaurant,Doctor's Office,Falafel Restaurant
34,Sutton Place,Grocery Store,Gym,Beer Garden,Greek Restaurant,Beer Store,Gym / Fitness Center,Food Court,Bakery,Bagel Shop,Indian Restaurant
35,Turtle Bay,Karaoke Bar,Coffee Shop,Tourist Information Center,Sushi Restaurant,Cheese Shop,Tennis Court,Farmers Market,Greek Restaurant,Grocery Store,Duty-free Shop


##### Cluster 4 (Tourist) #

In [33]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Upper West Side,American Restaurant,Italian Restaurant,Bakery,Tiki Bar,Southern / Soul Food Restaurant,Chinese Restaurant,Bookstore,Movie Theater,Juice Bar,Greek Restaurant
13,Lincoln Square,Theater,Indie Movie Theater,Concert Hall,Performing Arts Venue,Gym / Fitness Center,College Arts Building,Circus,Opera House,Fountain,Library
21,Tribeca,American Restaurant,Park,Yoga Studio,Sushi Restaurant,Indie Theater,Italian Restaurant,Greek Restaurant,Dog Run,Cycle Studio,Poke Place
24,West Village,Italian Restaurant,Cocktail Bar,Coffee Shop,Gourmet Shop,Austrian Restaurant,Speakeasy,Candy Store,Mediterranean Restaurant,Boutique,Board Shop
27,Gramercy,Pizza Place,Coffee Shop,Comedy Club,Spa,Liquor Store,Sushi Restaurant,Gourmet Shop,Beer Bar,Cocktail Bar,Mexican Restaurant
38,Flatiron,Japanese Restaurant,Furniture / Home Store,Cycle Studio,Salon / Barbershop,Salad Place,Russian Restaurant,Miscellaneous Shop,Donut Shop,Gym,Gym / Fitness Center


##### Cluster 4 (Artistic) #

In [35]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Chinese Restaurant,Sandwich Place,Greek Restaurant,Noodle House,Bubble Tea Shop,Museum,Bike Shop,Salon / Barbershop,New American Restaurant,Cocktail Bar
4,Hamilton Heights,Yoga Studio,Mexican Restaurant,Cocktail Bar,Bakery,Smoke Shop,Café,Mediterranean Restaurant,Caribbean Restaurant,Coffee Shop,Burger Joint
9,Yorkville,Wine Shop,Deli / Bodega,Italian Restaurant,Gym,Park,Sushi Restaurant,Dog Run,Liquor Store,Bagel Shop,Coffee Shop
11,Roosevelt Island,Deli / Bodega,Gym,Sandwich Place,Food & Drink Shop,Soccer Field,Outdoors & Recreation,Restaurant,Greek Restaurant,Residential Building (Apartment / Condo),School
16,Murray Hill,Japanese Restaurant,Burger Joint,Coffee Shop,Hotel,Shanghai Restaurant,Museum,Sushi Restaurant,Sandwich Place,Tea Room,Grocery Store
18,Greenwich Village,Italian Restaurant,French Restaurant,Café,Sushi Restaurant,Yoga Studio,Coffee Shop,Snack Place,Caribbean Restaurant,Gourmet Shop,Beer Bar
19,East Village,Korean Restaurant,Vietnamese Restaurant,Pet Café,Beer Store,Dog Run,Beer Bar,Bar,Moroccan Restaurant,Bagel Shop,Scandinavian Restaurant
22,Little Italy,Coffee Shop,Wine Bar,Ice Cream Shop,Café,Sandwich Place,Thai Restaurant,Spanish Restaurant,Chinese Restaurant,Salon / Barbershop,Salad Place
26,Morningside Heights,Bookstore,Park,American Restaurant,Greek Restaurant,Outdoor Sculpture,Mexican Restaurant,Farmers Market,Café,Food Truck,Sandwich Place
31,Noho,Wine Shop,Rock Club,French Restaurant,Hotel,Gym,Deli / Bodega,Coffee Shop,Cocktail Bar,Sandwich Place,Southern / Soul Food Restaurant
